#SparkSQL

Working with PySpark's built-in SQL compliant functionality to investigate flights-delay data at scale


*   Understand the limitations of SparkSQL
*   Experiment with createOrReplaceGlobalView
*   Work on same executions through pyspark methods



In [ ]:
#Download Data from source
Data Source Repo - https://kloudbitbucket.s3.amazonaws.com/krunal_ds/departuredelays.csv

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=fb3380ba9d15dabc94c9b062ab297c0eb9066685af601c2f9b06deb079f9d696
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
#Import necessary spark components
from pyspark.sql import *
from pyspark.sql.functions import *


In [3]:
#Create a SparkSession
spark = SparkSession.builder.appName("sqlpractice").getOrCreate()

In [8]:
#Import Data into Spark Native Dataframe
df = spark.read.option("Header","True").csv("/content/departuredelays.csv")

In [17]:
#Convert into an SQL complaint format
sdf = df
sdf.createOrReplaceTempView("delays")

In [23]:
#Display 100 rows of data with SQL query
spark.sql("select * from delays limit 100").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
|01030605|    0|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        ATL|
|01050605|    9|     602|   ABE|        ATL|
|01061215|   -6|     602|   ABE|        ATL|
|01061725|   69|     602|   ABE|        ATL|
|01061230|    0|     369|   ABE|        DTW|
|01060625|   -3|     602|   ABE|        ATL|
|01070600|    0|     369|   ABE|        DTW|
|01071725|    0|     602|   ABE|        ATL|
|01071230|    0|     369|   ABE|        DTW|
|01070625|    0|     602|   ABE|        ATL|
|01071219|    0|     569|   ABE|        ORD|
|01080600|

In [24]:
#Get a list of all Origin and Destination airports
spark.sql("select distinct origin,destination from delays").show()

+------+-----------+
|origin|destination|
+------+-----------+
|   ATL|        GSP|
|   BQN|        MCO|
|   CLE|        SJU|
|   DSM|        EWR|
|   EWR|        STT|
|   FSD|        ATL|
|   LAS|        LIT|
|   LBB|        DEN|
|   MCI|        IAH|
|   MCI|        MKE|
|   MDW|        MEM|
|   ORD|        PDX|
|   PBI|        DCA|
|   PHL|        MCO|
|   ROC|        CLE|
|   SJC|        LIH|
|   SMF|        BUR|
|   SNA|        PHX|
|   AUS|        ELP|
|   BMI|        MCO|
+------+-----------+
only showing top 20 rows



In [27]:
#Find the top 5 longest distance travel routes
spark.sql("select * from delays order by distance desc limit 5").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|02010751|   23|     999|   DSM|        PHX|
|01071459|   11|     999|   DSM|        PHX|
|02030751|    6|     999|   DSM|        PHX|
|01090751|    5|     999|   DSM|        PHX|
|02011459|   -1|     999|   DSM|        PHX|
+--------+-----+--------+------+-----------+



In [51]:
#Find top 3 origin airport with maximum flight delay occurances
spark.sql(
"""
select origin
from delays
group by origin
order by count(delay) desc limit 3
""").show()

+------+
|origin|
+------+
|   ATL|
|   DFW|
|   ORD|
+------+



In [33]:
# prompt: Find top 3 origin airport with maximum flight delay occurances what is the meaning odf it

spark.sql("""
SELECT origin, COUNT(*) as num_delays
FROM delays
WHERE delay > 0
GROUP BY origin
ORDER BY num_delays DESC LIMIT 3
""").show()

# This SQL query finds the top 3 origin airports with the most flight delays.
# It counts the number of delays for each origin airport (where delay is greater than 0), groups the results by origin,
# orders them in descending order based on the count, and limits the output to the top 3.
# This helps identify the airports that experience the most flight delays.


+------+----------+
|origin|num_delays|
+------+----------+
|   ATL|     35209|
|   ORD|     22867|
|   DEN|     20820|
+------+----------+



In [40]:
#Find top 3 destination airport with maximum flight delay occurances
spark.sql(
    """
    select origin,destination,max(delay) as delay_max
    from delays
    group by origin,destination
    order by delay_max desc
    limit 1
    """
).show()

+------+-----------+---------+
|origin|destination|delay_max|
+------+-----------+---------+
|   SMF|        SLC|      995|
+------+-----------+---------+



In [49]:
#Find the top three routes with maximum time-delay
spark.sql("select* from delays order by delay desc limit 3").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01090600|  995|     462|   SMF|        SLC|
|01200645|  993|     525|   MOT|        DEN|
|02061431|   99|     347|   DCA|        BOS|
+--------+-----+--------+------+-----------+



In [50]:
#Find the distance for top three max time-delay routes
spark.sql("select * from delays order by distance desc limit 3").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|02010751|   23|     999|   DSM|        PHX|
|01071459|   11|     999|   DSM|        PHX|
|02020751|  120|     999|   DSM|        PHX|
+--------+-----+--------+------+-----------+

